# Bài thực hành số 8: Dự đoán khách hàng rời ngân hàng (Churn Prediction) với PySpark

**Mục tiêu:** Sử dụng các lớp cơ bản của ML: Transformer, Estimator, Pipeline để xây dựng mô hình học máy và đánh giá mô hình.

**Nội dung:**
1. Đọc dữ liệu `Churn_Modelling.csv`.
2. Khám phá và trực quan hóa dữ liệu (EDA).
3. Tiền xử lý dữ liệu (xử lý biến phân loại, tạo vector đặc trưng).
4. Xây dựng và đánh giá mô hình Logistic Regression.
5. Tạo Pipeline, lưu và tải lại mô hình.
6. Thử nghiệm lựa chọn thuộc tính.
7. Tinh chỉnh tham số (Hyperparameter Tuning).


In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql.functions import col
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import shutil

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("ChurnPrediction_Lab8") \
    .getOrCreate()

# Giảm mức độ log để màn hình gọn hơn
spark.sparkContext.setLogLevel("ERROR")

print("Spark version:", spark.version)

## a) Sử dụng PySpark để đọc dữ liệu vào DataFrame

In [ ]:
# Đọc dữ liệu
df = spark.read.csv("Churn_Modelling.csv", header=True, inferSchema=True)

# Hiển thị 5 dòng đầu
df.show(5)

# In sơ đồ dữ liệu (schema)
df.printSchema()

## b) Thực hiện một số thống kê, trực quan hóa để hiểu dữ liệu

In [ ]:
# Thống kê mô tả
df.describe().show()

# Kiểm tra sự phân bố của biến mục tiêu 'Exited'
exited_count = df.groupBy("Exited").count().toPandas()

plt.figure(figsize=(6,4))
sns.barplot(x="Exited", y="count", data=exited_count)
plt.title("Phân bố khách hàng rời bỏ (Exited)")
plt.show()

# Ma trận tương quan (Correlation Matrix) - Chỉ lấy các cột số
numeric_cols = ["CreditScore", "Age", "Tenure", "Balance", "NumOfProducts", "HasCrCard", "IsActiveMember", "EstimatedSalary", "Exited"]
pdf = df.select(numeric_cols).toPandas()

plt.figure(figsize=(10,8))
sns.heatmap(pdf.corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Ma trận tương quan giữa các biến số")
plt.show()

## c) Tiền xử lý dữ liệu
Bao gồm:
- Loại bỏ cột `RowNumber`, `CustomerId`, `Surname`.
- Chuyển đổi giá trị chuỗi thành số (`StringIndexer`).
- Mã hóa One-Hot (`OneHotEncoder`) cho các biến `Geography`, `Gender`.
- Chuyển đổi các biến độc lập thành vector (`VectorAssembler`).


In [ ]:
# Loại bỏ các cột không cần thiết
data = df.drop("RowNumber", "CustomerId", "Surname")

# Xác định các cột phân loại và cột số
categorical_cols = ["Geography", "Gender"]
numeric_cols = ["CreditScore", "Age", "Tenure", "Balance", "NumOfProducts", "HasCrCard", "IsActiveMember", "EstimatedSalary"]

stages = []

# Xử lý các biến phân loại
for categoricalCol in categorical_cols:
    # Chuyển đổi chuỗi thành chỉ số số
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    # Mã hóa One-Hot
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    # Thêm vào các bước pipeline
    stages += [stringIndexer, encoder]

# Tạo vector đặc trưng (features vector)
# Đầu vào của Assembler bao gồm các vector one-hot của biến phân loại và các cột số
assemblerInputs = [c + "classVec" for c in categorical_cols] + numeric_cols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

print("Các bước tiền xử lý đã được định nghĩa.")

## d) Chia dữ liệu thành tập huấn luyện và tập kiểm tra (70/30)

In [ ]:
# Chia dữ liệu: 70% huấn luyện, 30% kiểm tra
train_data, test_data = data.randomSplit([0.7, 0.3], seed=42)

print(f"Số lượng mẫu tập huấn luyện: {train_data.count()}")
print(f"Số lượng mẫu tập kiểm tra: {test_data.count()}")

## e) Sử dụng Logistic Regression để huấn luyện mô hình
Chúng ta sẽ thêm Logistic Regression vào cuối Pipeline.


In [ ]:
# Khởi tạo mô hình Logistic Regression
lr = LogisticRegression(labelCol="Exited", featuresCol="features")

# Thêm mô hình vào danh sách stages
stages_lr = stages + [lr]

# Tạo Pipeline
pipeline = Pipeline(stages=stages_lr)

# Huấn luyện mô hình
print("Đang huấn luyện mô hình...")
model = pipeline.fit(train_data)
print("Huấn luyện hoàn tất.")

# Dự đoán trên tập kiểm tra
predictions = model.transform(test_data)

# Hiển thị kết quả dự đoán
predictions.select("prediction", "Exited", "features").show(5)

## f) Đánh giá hiệu suất mô hình
Sử dụng: Độ chính xác (Accuracy), F1 Score, Weighted Precision/Recall và AUC.


In [ ]:
# BinaryClassificationEvaluator cho AUC
binary_evaluator = BinaryClassificationEvaluator(labelCol="Exited", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
auc = binary_evaluator.evaluate(predictions)
print(f"Area Under ROC (AUC): {auc:.4f}")

# MulticlassClassificationEvaluator cho các chỉ số khác
multi_evaluator = MulticlassClassificationEvaluator(labelCol="Exited", predictionCol="prediction")

accuracy = multi_evaluator.evaluate(predictions, {multi_evaluator.metricName: "accuracy"})
f1 = multi_evaluator.evaluate(predictions, {multi_evaluator.metricName: "f1"})
recall = multi_evaluator.evaluate(predictions, {multi_evaluator.metricName: "weightedRecall"})
precision = multi_evaluator.evaluate(predictions, {multi_evaluator.metricName: "weightedPrecision"})

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Weighted Recall: {recall:.4f}")
print(f"Weighted Precision: {precision:.4f}")

## g), h), i) Pipeline, Lưu và Mở lại mô hình
Mô hình `model` ở trên đã là một PipelineModel (bao gồm cả các bước tiền xử lý). Chúng ta sẽ lưu nó lại và thử tải lên để dự đoán.


In [ ]:
model_path = "churn_logistic_regression_model"

# Kiểm tra nếu thư mục tồn tại thì xóa đi để lưu mới
if os.path.exists(model_path):
    shutil.rmtree(model_path)

# Lưu mô hình
model.save(model_path)
print(f"Đã lưu mô hình tại: {model_path}")

# Mở lại mô hình đã lưu
loaded_model = PipelineModel.load(model_path)
print("Đã tải lại mô hình.")

# Dự đoán lại cho dữ liệu test bằng mô hình vừa tải
new_predictions = loaded_model.transform(test_data)
new_auc = binary_evaluator.evaluate(new_predictions)
print(f"AUC của mô hình sau khi tải lại: {new_auc:.4f}")

## Lựa chọn thuộc tính xây dựng mô hình
Dựa trên phân tích tương quan ở bước EDA, chúng ta thấy `Age`, `Balance` và `IsActiveMember` có tương quan đáng kể hơn các thuộc tính khác.
Chúng ta sẽ thử xây dựng mô hình chỉ với các thuộc tính này (cùng với các biến phân loại) để xem hiệu suất thay đổi như thế nào.


In [ ]:
# Chọn các thuộc tính số quan trọng hơn (ví dụ minh họa)
selected_numeric_cols = ["Age", "Balance", "IsActiveMember"]

# Định nghĩa lại các bước pipeline cho tập thuộc tính mới
stages_selected = []

# Xử lý lại biến phân loại (như cũ)
for categoricalCol in categorical_cols:
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages_selected += [stringIndexer, encoder]

# VectorAssembler mới
assemblerInputs_selected = [c + "classVec" for c in categorical_cols] + selected_numeric_cols
assembler_selected = VectorAssembler(inputCols=assemblerInputs_selected, outputCol="features")
stages_selected += [assembler_selected]

# Logistic Regression
lr_selected = LogisticRegression(labelCol="Exited", featuresCol="features")
stages_selected += [lr_selected]

# Tạo và huấn luyện pipeline mới
pipeline_selected = Pipeline(stages=stages_selected)
model_selected = pipeline_selected.fit(train_data)

# Đánh giá
pred_selected = model_selected.transform(test_data)
auc_selected = binary_evaluator.evaluate(pred_selected)
print(f"AUC (Mô hình với thuộc tính chọn lọc): {auc_selected:.4f}")
print(f"AUC (Mô hình ban đầu): {auc:.4f}")

## Thay đổi tham số của mô hình (Hyperparameter Tuning)
Chúng ta sẽ tinh chỉnh các tham số `maxIter` và `regParam` của Logistic Regression để tìm ra cấu hình tốt nhất.


In [ ]:
# Khởi tạo Logistic Regression mới cho việc tuning
lr_tuning = LogisticRegression(labelCol="Exited", featuresCol="features")

# Tái sử dụng các bước tiền xử lý ban đầu (stages[0] đến stages[2] là pre-processing)
pipeline_tuning = Pipeline(stages=stages + [lr_tuning])

# Tạo lưới tham số
paramGrid = ParamGridBuilder() \
    .addGrid(lr_tuning.maxIter, [10, 20]) \
    .addGrid(lr_tuning.regParam, [0.01, 0.1]) \
    .build()

# Cross Validator (Kiểm định chéo)
crossval = CrossValidator(estimator=pipeline_tuning,
                          estimatorParamMaps=paramGrid,
                          evaluator=binary_evaluator,
                          numFolds=3) # 3-fold cv

print("Đang thực hiện Cross-Validation (có thể mất một chút thời gian)...")
cvModel = crossval.fit(train_data)

# Lấy mô hình tốt nhất
best_model = cvModel.bestModel
print("Tuning hoàn tất.")

# Đánh giá mô hình tốt nhất
tuning_predictions = best_model.transform(test_data)
auc_tuning = binary_evaluator.evaluate(tuning_predictions)

print(f"AUC (Mô hình sau khi tinh chỉnh): {auc_tuning:.4f}")
print(f"Tham số RegParam tốt nhất: {best_model.stages[-1].getRegParam()}")
print(f"Tham số MaxIter tốt nhất: {best_model.stages[-1].getMaxIter()}")

### Kết luận
Qua bài thực hành, chúng ta đã đi qua quy trình đầy đủ từ đọc dữ liệu, xử lý, huấn luyện, đánh giá, lưu trữ và tinh chỉnh mô hình dự đoán rời bỏ ngân hàng sử dụng PySpark.
